In [46]:
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as dataset
from torchvision import transforms

import torchvision.models as models

import torch
import torch.nn as nn

from torch import autograd, optim

import time
import glob
import os
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, random_split


In [47]:
path = './data/'

In [48]:
phase = ['train', 'test']

In [49]:
for p in phase:
    files = glob.glob(os.path.join(path+p, '*/*.jpg'))

In [50]:
files = glob.glob(os.path.join(path+p, '*/*.jpg'))

In [51]:
print(len(files))

2023


In [52]:
no_of_images =len(files)

In [53]:
shuffle = np.random.permutation(no_of_images)

In [54]:
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485,0.465,0.406),(0.229,0.224,0.225))
                                                ])

In [56]:
train_dataset = dataset.ImageFolder(path+'train', transform)
valid_dataset = dataset.ImageFolder(path+'valid', transform)

In [57]:
dataset = ConcatDataset([train_dataset, valid_dataset])

In [29]:
train_size = len(train_dataset)

In [40]:
train_ds, valid_ds = random_split(train_dataset, [train_size-200, 200])

In [59]:
print(len(train_dataset))

6005


In [13]:
train_dataloader = DataLoader(train_dataset,batch_size= 16, shuffle= True)
valid_dataloader = DataLoader(valid_dataset,batch_size= 16, shuffle= True)

# Define Network

In [14]:
model  = models.resnet18(pretrained=True)

In [15]:
num_ftrs = model.fc.in_features

In [16]:
model.fc = nn.Linear(num_ftrs, 2)

In [17]:
# Run the model on the GPU
model = model.cuda()

In [18]:
#define loss function
crit = nn.CrossEntropyLoss()

In [19]:
# define the optimizer
opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9 )

In [20]:
#Helps to dynamically change the learning rate
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size = 7, gamma=0.1)

# Train the model

In [21]:
epochs = 25

In [23]:
epochs = 5
min_valid_loss = np.inf

for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    
    for i,(data,labels) in enumerate (train_dataloader):
        image = data.cuda()
        label = labels.cuda()
         
        # Clear the gradients    
        model.zero_grad()
        
        #forward
        outputs = model(image)
        _, preds = torch.max(outputs.data, 1)
        loss = crit(outputs, label)
        
        loss.backward() #calculates the gradients quentity 
        opt.step()#save changes
        running_loss += loss.item()
        
        
    valid_loss = 0.0
    model.eval()
    
    for i, (data, labels) in enumerate(valid_dataloader):
        image = data.cuda()
        label = labels.cuda()
        
        target = model(image)
        loss = crit(target, label)
        valid_loss = loss.item() * data.size(0)
         
    print(f'Epoch {epoch+1} \t\t Training Loss: {running_loss / len(train_dataloader)} \t\t Validation Loss: {valid_loss / len(valid_dataloader)}')
    
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')
    

Epoch 1 		 Training Loss: 0.1005421936427778 		 Validation Loss: 0.00017408856749534608
Validation Loss Decreased(inf--->0.021761) 	 Saving The Model


KeyboardInterrupt: 

# Add Cross_validation

In [60]:
from sklearn.model_selection import KFold

#do the training for different 5 iterations
k_folds = 4

# For fold results
results = {}

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)


In [63]:
epochs = 5
min_valid_loss = np.inf

for fold, (train_ids, val_ids) in enumerate((kfold.split(dataset))):
  # Print
  print('----------------------------------------------------------------------------')
  print(f'----------------------------FOLD {fold}------------------------------------')
  print('----------------------------------------------------------------------------')
        
  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)


  # Define data loaders for training and testing data in this fold
  train_dataloader = DataLoader(dataset, batch_size=16, sampler=train_subsampler)
  valid_dataloader = DataLoader(dataset, batch_size=1, sampler=val_subsampler)
  
  for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    
    for i,(data,labels) in enumerate (train_dataloader):
        image = data.cuda()
        label = labels.cuda()
         
        # Clear the gradients    
        model.zero_grad()
        
        #forward
        outputs = model(image)
        _, preds = torch.max(outputs.data, 1)
        loss = crit(outputs, label)
        
        loss.backward() #calculates the gradients quentity 
        opt.step()#save changes
        running_loss += loss.item()
        
    valid_loss = 0.0
    model.eval()
    for i, (data, labels) in enumerate(valid_dataloader):
        image = data.cuda()
        label = labels.cuda()
        
        target = model(image)
        loss = crit(target, label)
        valid_loss = loss.item() * data.size(0)
         
    print(f'Epoch {epoch+1} \t\t Training Loss: {running_loss / len(train_dataloader)} \t\t Validation Loss: {valid_loss / len(valid_dataloader)}')
    
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')
    

FOLD 0
--------------------------------
Epoch 1 		 Training Loss: 0.12147235691837194 		 Validation Loss: 1.0004269887696017e-06
Validation Loss Decreased(inf--->0.002003) 	 Saving The Model
Epoch 2 		 Training Loss: 0.04655375749419537 		 Validation Loss: 3.7560004510216186e-06
Epoch 3 		 Training Loss: 0.0330657532170122 		 Validation Loss: 2.179303038218705e-08
Validation Loss Decreased(0.002003--->0.000044) 	 Saving The Model
Epoch 4 		 Training Loss: 0.02549827255277589 		 Validation Loss: 2.0818581949309124e-07
Epoch 5 		 Training Loss: 0.023994107772639526 		 Validation Loss: 1.908817605769539e-06
FOLD 1
--------------------------------
Epoch 1 		 Training Loss: 0.04762444016163732 		 Validation Loss: 2.210178301201436e-08
Epoch 2 		 Training Loss: 0.019528194136155627 		 Validation Loss: 1.3598506904390084e-06
Epoch 3 		 Training Loss: 0.011678242226683691 		 Validation Loss: 5.957485379577905e-11
Validation Loss Decreased(0.000044--->0.000000) 	 Saving The Model
Epoch 4 		 Tra